# Import elk snapshot in another fabric slice

## Import

In [1]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

MFLib version  1.0.0 


In [2]:
import importlib
import mf_data_transfer
importlib.reload(mf_data_transfer)
from mf_data_transfer import mf_data_import

## Slice Info

In [3]:
slice_name = "backup-testing"
node1_name = 'data-backup'

In [4]:
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    node = slice.get_node(name=node1_name)
except Exception as e:
    print(f"Fail: {e}")

-----------  ------------------------------------
Slice Name   backup-testing
Slice ID     a51c0f31-82a8-46b1-b19d-5e9216016dbf
Slice State  StableOK
Lease End    2023-06-13 00:02:12 +0000
-----------  ------------------------------------


## Start elk container to use the snapshot

### upload elasticsearch.yml to the node

In [5]:
# Upload an example elasticsearch.yml file
node.upload_file(local_file_path="/home/fabric/work/data_transfer/elk_docker/elasticsearch.yml", 
                 remote_file_path='/home/ubuntu/elasticsearch.yml')
cmd = "sudo chown root:root /home/ubuntu/elasticsearch.yml"
stdout, stderr=node.execute(cmd)

 sudo: unable to resolve host data-backup: Name or service not known
 

### create docker volumes

In [6]:
cmd = "sudo docker volume create elk_snapshotbackup"
stdout, stderr=node.execute(cmd)
cmd = "sudo docker volume create elk_data01"
stdout, stderr=node.execute(cmd)

elk_snapshotbackup
 sudo: unable to resolve host data-backup: Name or service not known
 elk_data01
 sudo: unable to resolve host data-backup: Name or service not known
 

### Start the container

In [7]:
cmd = f'''
        sudo docker run -dit --name elasticsearch \
        --user 0:0 \
        -p 9200:9200 \
        -e "discovery.type=single-node" \
        -e "path.repo=/usr/share/elasticsearch/backup" \
        -v /home/ubuntu/elasticsearch.yml:/usr/share/elasticsearch/config/elasticsearch.yml \
        -v elk_data01:/usr/share/elasticsearch/data \
        -v elk_snapshotbackup:/usr/share/elasticsearch/backup \
        --ulimit memlock=-1:-1 \
        --log-driver json-file \
        elasticsearch:7.13.0
      '''
stdout, stderr=node.execute(cmd)

f1d11bee74183760255c9e20116da0f77eccc1f924dc915b36994dd624ad8e16
 sudo: unable to resolve host data-backup: Name or service not known
Unable to find image 'elasticsearch:7.13.0' locally
7.13.0: Pulling from library/elasticsearch
7a0437f04f83: Pulling fs layer
dea920879f70: Pulling fs layer
45def7f6c6b6: Pulling fs layer
ff173f2d6e01: Pulling fs layer
96cca0727b1d: Pulling fs layer
67c1b86afe55: Pulling fs layer
007b3c569f35: Pulling fs layer
ff173f2d6e01: Waiting
96cca0727b1d: Waiting
67c1b86afe55: Waiting
007b3c569f35: Waiting
45def7f6c6b6: Verifying Checksum
45def7f6c6b6: Download complete
dea920879f70: Verifying Checksum
dea920879f70: Download complete
96cca0727b1d: Verifying Checksum
96cca0727b1d: Download complete
7a0437f04f83: Verifying Checksum
7a0437f04f83: Download complete
67c1b86afe55: Verifying Checksum
67c1b86afe55: Download complete
007b3c569f35: Verifying Checksum
007b3c569f35: Download complete
7a0437f04f83: Pull complete
ff173f2d6e01: Verifying Checksum
ff173f2d6e01: D

In [8]:
node.execute("sudo docker ps")

CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS                                                 NAMES
f1d11bee7418   elasticsearch:7.13.0      "/bin/tini -- /usr/l…"   19 seconds ago   Up 18 seconds   0.0.0.0:9200->9200/tcp, :::9200->9200/tcp, 9300/tcp   elasticsearch
f207407bdae2   prom/prometheus:v2.33.4   "/bin/prometheus --c…"   9 minutes ago    Up 9 minutes    0.0.0.0:9090->9090/tcp, :::9090->9090/tcp             prometheus
43ca34ee7daf   data_transfer:1.0         "python3"                16 minutes ago   Up 16 minutes                                                         data_transfer
 sudo: unable to resolve host data-backup: Name or service not known
 

('CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS                                                 NAMES\nf1d11bee7418   elasticsearch:7.13.0      "/bin/tini -- /usr/l…"   19 seconds ago   Up 18 seconds   0.0.0.0:9200->9200/tcp, :::9200->9200/tcp, 9300/tcp   elasticsearch\nf207407bdae2   prom/prometheus:v2.33.4   "/bin/prometheus --c…"   9 minutes ago    Up 9 minutes    0.0.0.0:9090->9090/tcp, :::9090->9090/tcp             prometheus\n43ca34ee7daf   data_transfer:1.0         "python3"                16 minutes ago   Up 16 minutes                                                         data_transfer\n',
 'sudo: unable to resolve host data-backup: Name or service not known\n')

### Change snapshot dir permission

In [9]:
cmd = f"sudo chown -R 1000:1000 /var/lib/docker/volumes/elk_snapshotbackup/_data"
stdout, stderr=node.execute(cmd)

 sudo: unable to resolve host data-backup: Name or service not known
 

### Register a backup repository

In [10]:
repo_name = "backup_repository"
default_dir = "/usr/share/elasticsearch/backup"
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}?pretty" -H "Content-Type: application/json" -d \'{{ "type": "fs", "settings": {{ "location": "{default_dir}" }} }}\''
stdout, stderr=node.execute(cmd)

{
  "acknowledged" : true
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   107  100    28  100    79    166    470 --:--:-- --:--:-- --:--:--   636
 

### Create a snapshot

In [11]:
snapshot_name = "snapshot_test"
json_str = '"ignore_unavailable": true, "include_global_state": false'
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?wait_for_completion=true&pretty" -H "Content-Type: application/json" -d \'{json_str}\''
stdout, stderr= node.execute(cmd)

{
  "snapshot" : {
    "snapshot" : "snapshot_test",
    "uuid" : "7J0aqwEERLW99AfyRPRjgw",
    "version_id" : 7130099,
    "version" : "7.13.0",
    "indices" : [ ],
    "data_streams" : [ ],
    "include_global_state" : true,
    "state" : "SUCCESS",
    "start_time" : "2023-05-30T00:33:27.585Z",
    "start_time_in_millis" : 1685406807585,
    "end_time" : "2023-05-30T00:33:27.585Z",
    "end_time_in_millis" : 1685406807585,
    "duration_in_millis" : 0,
    "failures" : [ ],
    "shards" : {
      "total" : 0,
      "failed" : 0,
      "successful" : 0
    },
    "feature_states" : [ ]
  }
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   659  100   602  100    57   5234    495 --:--:-- --:--:-- --:--:--  5730
 

### Replace the file with the tar file

In [12]:
snapshot_dir = "/var/lib/docker/volumes/elk_snapshotbackup/_data"
cmd = f"sudo rm -rf {snapshot_dir}"
stdout, stderr= node.execute(cmd)

 sudo: unable to resolve host data-backup: Name or service not known
 

In [13]:
# untar backup snapshot files to snapshot dir
snapshot_parent_dir = "/var/lib/docker/volumes/elk_snapshotbackup/"
elk_tar_file_name = "elk_backup_0529.tar"
elk_tar_path = "/home/ubuntu/elk/download"
cmd = f'sudo tar -xvf {elk_tar_path}/{elk_tar_file_name} -C {snapshot_parent_dir}'
node.execute(cmd, quiet=True)

('_data/\n_data/meta-QzNC8U7WTG-hSSuajanuJQ.dat\n_data/snap-QzNC8U7WTG-hSSuajanuJQ.dat\n_data/indices/\n_data/indices/w2asDBdJRU-kq93pQsIDOA/\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__94aROeo0Sjm62PjidM3VTw\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__QmWIzGKUTXirzDoxkpS9Pw\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__wdGCTzkfSgulirLO83DefQ\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__2wCu4DuERCC2Ih9m_N0wCg\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__vNbia6ooQU63DhAvBMeNTA\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__3Zkv-jkCSxGVCFM4N6Oi7Q\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__11VRQyf2QlecgGy4nB9FRw\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/snap-QzNC8U7WTG-hSSuajanuJQ.dat\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__09hQxcm6SkmNb6H0nJmSLQ\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__POfpX22SR1musJMJtYKxWw\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__8iKrb6GpRyq26-lbe3YnoQ\n_data/indices/w2asDBdJRU-kq93pQsIDOA/0/__RXBIgGcHT8eNi3HWs5a6XQ\n_data/indices/w2asDBdJRU-kq93

In [16]:
cmd = f"sudo chown -R 1000:1000 /var/lib/docker/volumes/elk_snapshotbackup/_data"
stdout, stderr=node.execute(cmd)
cmd = f"sudo ls {snapshot_parent_dir}/_data"
node.execute(cmd, quiet=True)

 sudo: unable to resolve host data-backup: Name or service not known
 

('index-0\nindex.latest\nindices\nmeta-QzNC8U7WTG-hSSuajanuJQ.dat\nsnap-QzNC8U7WTG-hSSuajanuJQ.dat\n',
 'sudo: unable to resolve host data-backup: Name or service not known\n')

### Restart the container

In [17]:
cmd = "sudo docker restart elasticsearch"
stdout, stderr= node.execute(cmd)

elasticsearch
 sudo: unable to resolve host data-backup: Name or service not known
 

### Check snapshot

In [19]:
# If you dont see the snapshot, you need to restart the container again(above step)
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f"curl -X GET 'http://localhost:9200/_cat/snapshots/{repo_name}?v'"
stdout, stderr= node.execute(cmd)

id             status start_epoch start_time end_epoch  end_time duration indices successful_shards failed_shards total_shards
snapshot_test SUCCESS 1685404519  23:55:19   1685404519 23:55:19    801ms       9                 9             0            9
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   254  100   254    0     0   1788      0 --:--:-- --:--:-- --:--:--  1788
 

In [20]:
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= node.execute(cmd)

{
  "snapshots" : [
    {
      "snapshot" : "snapshot_test",
      "uuid" : "QzNC8U7WTG-hSSuajanuJQ",
      "version_id" : 7130099,
      "version" : "7.13.0",
      "indices" : [
        "packetbeat-7.13.2-2023.05.29-000001",
        ".kibana-event-log-7.13.0-000001",
        ".apm-custom-link",
        ".ds-ilm-history-5-2023.05.29-000001",
        ".apm-agent-configuration",
        "filebeat-7.13.2-2023.05.29-000001",
        ".kibana_7.13.0_001",
        "metricbeat-7.13.2-2023.05.29-000001",
        ".kibana_task_manager_7.13.0_001"
      ],
      "data_streams" : [
        "ilm-history-5"
      ],
      "include_global_state" : false,
      "state" : "SUCCESS",
      "start_time" : "2023-05-29T23:55:19.031Z",
      "start_time_in_millis" : 1685404519031,
      "end_time" : "2023-05-29T23:55:19.832Z",
      "end_time_in_millis" : 1685404519832,
      "duration_in_millis" : 801,
      "failures" : [ ],
      "shards" : {
        "total" : 9,
        "failed" : 0,
        "success

### Delete indice if any

In [21]:
cmd = f'curl -X DELETE "http://localhost:9200/_all"'
stdout, stderr= node.execute(cmd)

{"acknowledged":true}   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    21  100    21    0     0   2333      0 --:--:-- --:--:-- --:--:--  2625
 

### Check indice

In [22]:
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= node.execute(cmd)

   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 

### Restore using the snapshot

In [23]:
# Need to specify indice names to restore
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "index_settings": {{ "index.number_of_replicas": 1 }} }}\''
#print (cmd)
stdout, stderr= node.execute(cmd)

{
  "accepted" : true
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   198  100    24  100   174     39    282 --:--:-- --:--:-- --:--:--   321
 

### Check progress

In [25]:
cmd = f'curl -XGET "http://localhost:9200/_cat/recovery?v"'
stdout, stderr= node.execute(cmd)

index                               shard time  type     stage source_host source_node target_host target_node  repository        snapshot      files files_recovered files_percent files_total bytes    bytes_recovered bytes_percent bytes_total translog_ops translog_ops_recovered translog_ops_percent
metricbeat-7.13.2-2023.05.29-000001 0     906ms snapshot done  n/a         n/a         172.17.0.3  f1d11bee7418 backup_repository snapshot_test 70    70              100.0%        70          26603074 26603074        100.0%        26603074    0            0                      100.0%
packetbeat-7.13.2-2023.05.29-000001 0     580ms snapshot done  n/a         n/a         172.17.0.3  f1d11bee7418 backup_repository snapshot_test 49    49              100.0%        49          3025386  3025386         100.0%        3025386     0            0                      100.0%
filebeat-7.13.2-2023.05.29-000001   0     632ms snapshot done  n/a         n/a         172.17.0.3  f1d11bee7418 backup_repositor

### After the process finishes, check the indice

In [26]:
# You should see the same indice as what you exported 
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= node.execute(cmd)

yellow open metricbeat-7.13.2-2023.05.29-000001 sJdL4EPMQy2yM-ZIvOZAdQ 1 1 61948 18 31.6mb 31.6mb
yellow open packetbeat-7.13.2-2023.05.29-000001 h1BjjPHGTumyO-Bkc_MKag 1 1  5946  0  2.8mb  2.8mb
yellow open filebeat-7.13.2-2023.05.29-000001   FiCGkrphRY-2pxfqc2cn1g 1 1  8865  0  2.3mb  2.3mb
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   294  100   294    0     0  12250      0 --:--:-- --:--:-- --:--:-- 12250
 

## Set up kibana

In [27]:
cmd = f'sudo docker run -d --name kibana --link elasticsearch:elasticsearch -p 5601:5601 docker.elastic.co/kibana/kibana:7.13.0'
stdout, stderr= node.execute(cmd)

254ddec1abbd2b42a90f0b6566ace1aa16f4657af4cbfc1c30725d904c942c19
 sudo: unable to resolve host data-backup: Name or service not known
Unable to find image 'docker.elastic.co/kibana/kibana:7.13.0' locally
7.13.0: Pulling from kibana/kibana
ddf49b9115d7: Pulling fs layer
f928d0217aa0: Pulling fs layer
7e03d650d77f: Pulling fs layer
1fee982cad48: Pulling fs layer
54b96c78839b: Pulling fs layer
1bdf6e4a0acd: Pulling fs layer
4f8d16de5081: Pulling fs layer
efddbc97b11f: Pulling fs layer
9fc7be64eb60: Pulling fs layer
fa97b9dbafef: Pulling fs layer
0b097a63c483: Pulling fs layer
dabb3421c1f8: Pulling fs layer
1535b8865a03: Pulling fs layer
1fee982cad48: Waiting
54b96c78839b: Waiting
1bdf6e4a0acd: Waiting
4f8d16de5081: Waiting
efddbc97b11f: Waiting
9fc7be64eb60: Waiting
fa97b9dbafef: Waiting
0b097a63c483: Waiting
dabb3421c1f8: Waiting
1535b8865a03: Waiting
7e03d650d77f: Verifying Checksum
7e03d650d77f: Download complete
1fee982cad48: Verifying Checksum
1fee982cad48: Download complete
f928d021

## View data in the web ui

##### Run 'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}' ; Then go to http://localhost:10030